In [6]:
import pandas as pd

# Örnek etiketli veri kümesi (manuel olarak oluşturulmuş)
labeled_data = pd.DataFrame({
    'Content': [
        "Küresel ısınma dünya çapında ciddi etkiler yaratıyor...",
        "Ekonomi bakanı yeni teşvik paketini açıkladı...",
        "Deniz seviyelerinin yükselmesi kıyı şehirlerini tehdit ediyor...",
        "Yeni bir güneş enerjisi santrali açıldı...",
        "Yerel halk, orman yangınlarına karşı önlem alıyor...",
        "Teknoloji şirketi yeni bir akıllı telefon tanıttı...",
        "Plastik atıkların okyanuslara zarar verdiği tespit edildi...",
        "Uluslararası anlaşmalar iklim değişikliğiyle mücadeleyi hedefliyor...",
        "Spor müsabakalarında büyük heyecan yaşandı...",
        "Yeni tarım teknikleri sürdürülebilirlik sağlıyor...",
        "Hava kirliliği insan sağlığını olumsuz etkiliyor...",
        "Fosil yakıtların kullanımının azaltılması gerektiği belirtiliyor...",
        "Ormanlar karbon emilimi için kritik öneme sahip...",
        "Yeni yürüyüş parkuru doğaseverlerin ilgisini çekiyor...",
        "İklim değişikliği nedeniyle kuraklık artıyor...",
        "Elektrikli araçların yaygınlaşması karbon ayak izini azaltıyor...",
        "Yerel yönetimler geri dönüşüm programlarını artırıyor...",
        "Yapay zeka alanında büyük ilerlemeler kaydedildi...",
        "Sanat galerisi yeni sergi açtı...",
        "Geleceğin enerji kaynakları yenilenebilir enerji olacak...",
        "Şehirde toplu taşıma sistemleri genişletiliyor...",
        "Küresel biyolojik çeşitlilik tehlikede...",
        "Çevre dostu binalar enerji tasarrufu sağlıyor...",
        "Uzay araştırmalarında yeni keşifler yapıldı...",
        "Geri dönüşüm fabrikası yeni tesis açtı...",
        "Ulusal parklar biyolojik çeşitliliği korumada önemli rol oynuyor...",
        "Orman yangınları doğal yaşamı tehdit ediyor...",
        "Temiz enerji teknolojileri hızla gelişiyor...",
        "Gıda israfını azaltma kampanyaları başlatıldı...",
        "Su kaynaklarının korunması için yeni düzenlemeler yapıldı..."
    ],
    'cevre_category': [
        1, 0, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 1
    ]
})
labeled_data2 = pd.read_csv("çevehaberlerikatogorizeedilmis.csv")
labeled_data2 = labeled_data2[['Content', 'cevre_category']]
labeled_data = pd.concat([labeled_data, labeled_data2], ignore_index=True)
print(labeled_data2)
# Eğitim ve doğrulama setlerine bölme
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    labeled_data['Content'], labeled_data['cevre_category'], test_size=0.2, random_state=42
)


                                               Content  cevre_category
0     ANTALYA - Sağlık ve Sigorta Yöneticileri Dern...               1
1    Meteoroloji Genel Müdürlüğü ( MGM, güncel hava...               1
2     fentanil, uyuşturucu krizinin üçüncü dalgası ...               0
3    Plastikleri hayatımızdan çıkarmanın önemi ile ...               1
4    Devir hızla değişiyor, etrafımızdaki her şey ç...               0
..                                                 ...             ...
295  \r\n\r\nCumhur İttifakı Bayraklı Belediye Başk...               0
296  \r\n\r\nBaşkan Recep Tayyip Erdoğan, "Bugüne k...               0
297  \r\n\r\nFethiye ve Marmaris'in körfezlerini ba...               1
298  İstanbul Büyükşehir Belediye Başkan Adayı Mura...               1
299  Mansur Yavaş, Bala ilçesinde Seçim Koordinasyo...               1

[300 rows x 2 columns]


In [12]:
from transformers import BertTokenizer,BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
model_name = "bert-base-multilingual-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2).to('cpu')


from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}
# Veri seti sınıfı
class NewsDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer(text, truncation=True, padding='max_length', max_length=512, return_tensors="pt")
        item = {key: val.squeeze() for key, val in inputs.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

# Eğitim ve doğrulama veri setleri
train_dataset = NewsDataset(train_texts.tolist(), train_labels.tolist(), tokenizer)
val_dataset = NewsDataset(val_texts.tolist(), val_labels.tolist(), tokenizer)

# Eğitim argümanları
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    no_cuda=True  # CUDA kullanmamak için ekleyin
)

# Trainer oluşturma
trainer = Trainer(
    model=model,
    compute_metrics=compute_metrics,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Modeli eğitme
trainer.train()



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.685631,0.545455,0.385027
2,No log,0.662624,0.545455,0.385027
3,No log,0.489715,0.757576,0.758021


TrainOutput(global_step=99, training_loss=0.6564562248461174, metrics={'train_runtime': 2010.76, 'train_samples_per_second': 0.394, 'train_steps_per_second': 0.049, 'total_flos': 208383955845120.0, 'train_loss': 0.6564562248461174, 'epoch': 3.0})

In [9]:
preds_output = trainer.predict(val_dataset)
preds_output.metrics

{'test_loss': 0.6536951661109924,
 'test_runtime': 44.7765,
 'test_samples_per_second': 1.474,
 'test_steps_per_second': 0.201}

In [30]:
import torch

input_text = "çevre iklim yağış sel deprem"



inputs = tokenizer(input_text, truncation=True, padding='max_length', max_length=512, return_tensors="pt")

# Dönüştürülmüş veriyi modele yükleyin
inputs = {key: val.to(trainer.args.device) for key, val in inputs.items()}

# Modelin tahmin yapmasını sağlayın
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Tahminleri alın
predicted_class = torch.argmax(logits, dim=-1).item()

# Tahmin sonuçlarını yazdırın
print("Tahmin Sınıfı:", predicted_class)

probabilities = torch.nn.functional.softmax(logits, dim=-1)
predicted_probability = probabilities[0][predicted_class].item()

print("Tahmin Yüzdesi:", predicted_probability)

Tahmin Sınıfı: 1
Tahmin Yüzdesi: 0.8770196437835693


In [33]:
import re
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm  

data = pd.read_csv("birlesik_son02_df.csv")  

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'\W', ' ', text)  
    text = text.lower()  
    return text

data['cleaned_text'] = data['content'].apply(clean_text)

unlabeled_dataset = NewsDataset(data['cleaned_text'], tokenizer=tokenizer)
# Dataset ve DataLoader
dataloader = DataLoader(unlabeled_dataset, batch_size=32, num_workers=4)  # Batch size ve num_workers değerlerini ayarlayın


device = torch.device('cuda')
model.to(device)

pseudo_labels = []

# Model tahminleri
model.eval()
with torch.no_grad():
    for batch in tqdm(dataloader):
        inputs = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        predicted_probabilities = probabilities.max(dim=-1).values
        predicted_classes = logits.argmax(dim=-1)
        
        for prob, pred_class in zip(predicted_probabilities, predicted_classes):
            if prob.item() > 0.70:  # 60% üzerinde güvenle tahmin
                pseudo_labels.append(pred_class.item())
            else:
                pseudo_labels.append(-1)  # Belirsiz tahminler için -1

data['pseudo_label'] = pseudo_labels


100%|█████████████████████████████████████████| 392/392 [20:33<00:00,  3.15s/it]


In [41]:
data[data['pseudo_label']==-1]

,source,content,date,cleaned_text,pseudo_label
3,gazeteduvar,"DUVAR - Türkiye Futbol Federasyonu, Galatasar...",11-3-2023,duvar türkiye futbol federasyonu galatasar...,-1
5,sozcu,"Şarkıcı ve sosyal medya fenomeni Banu Parlak,...",11-3-2023,şarkıcı ve sosyal medya fenomeni banu parlak ...,-1
6,gazeteduvar,"DUVAR - Artan kira fiyatları, metropollerin y...",11-3-2023,duvar artan kira fiyatları metropollerin y...,-1
9,aa,İstanbulda pazar günü düzenlenecek 45. İstanbu...,11-3-2023,i̇stanbulda pazar günü düzenlenecek 45 i̇stan...,-1
12,sozcu,Sakaryanın Hendek ilçesindeki bir inşaatta er...,11-3-2023,sakaryanın hendek ilçesindeki bir inşaatta er...,-1
...,...,...,...,...,...
12519,haberturk,"Bolu'da, kontrolden çıkarak şarampole devrilen...",05-27-2024,bolu da kontrolden çıkarak şarampole devrilen...,-1
12525,haberturk,"Olay, Konya'da meydana geldi.Kız arkadaşıyla m...",05-27-2024,olay konya da meydana geldi kız arkadaşıyla m...,-1
12526,haberturk,"Zafer Partisi Genel Başkanı Ümit Özdağ, Sandık...",05-27-2024,zafer partisi genel başkanı ümit özdağ sandık...,-1
12529,haberturk,"Edinilen bilgiye göre, Ankara İl Jandarma Komu...",05-27-2024,edinilen bilgiye göre ankara i̇l jandarma kom...,-1


In [34]:
data.to_csv("trainedilmis.csv")

In [ ]:
import re
# Etiketlenmemiş veri seti
data = pd.read_csv("birlesik_data.csv")  # lemmatize edilmemiş bütün veri olsun

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'\W', ' ', text)  
    text = text.lower()  
    return text

data['cleaned_text'] = data['content'].apply(clean_text)
unlabeled_dataset = NewsDataset(data['cleaned_text'], tokenizer=tokenizer)

# Pseudo-labeling: Etiketlenmemiş veri üzerinde tahmin yapma
pseudo_labels = []
for i in range(len(unlabeled_dataset)):
    inputs = unlabeled_dataset[i]
    inputs = {key: val.unsqueeze(0).to(trainer.args.device) for key, val in inputs.items()}
    with torch.no_grad():
        logits = model(**inputs).logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predicted_probability = probabilities[0][predicted_class].item()
    if 60 > int(predicted_probability):  
        pseudo_labels.append(logits.argmax(dim=-1).item())
data['pseudo_label'] = pseudo_labels

In [11]:
# Pseudo-label'ları etiketlenmemiş veri setine ekleme
pseudo_labeled_dataset = NewsDataset(unlabeled_dataset, pseudo_labels, tokenizer)

# Etiketli veri seti ile pseudo-labeled veri setini birleştirme
combined_texts = train_texts.tolist() + unlabeled_texts
combined_labels = train_labels.tolist() + pseudo_labels
combined_dataset = NewsDataset(combined_texts, combined_labels, tokenizer)

# Modeli tekrar eğitme
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=combined_dataset,
    eval_dataset=val_dataset
)

trainer.train()


NameError: name 'unlabeled_texts' is not defined

In [27]:
#del model
torch.cuda.empty_cache()

In [16]:
import torch
from torch.utils.data import DataLoader

# Define the batch size
batch_size = 16

unlabeled_dataloader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

pseudo_labels = []

# EVAL MODU MANTIKLI MI?
model.eval()

#GPU varsa
model.to(trainer.args.device)

# Disable gradient computation
with torch.no_grad():
    for batch in unlabeled_dataloader:
        
        inputs = {key: val.to(trainer.args.device) for key, val in batch.items()}
        
   
        outputs = model(**inputs)
        logits = outputs.logits
        
        
        batch_pseudo_labels = logits.argmax(dim=-1).cpu().numpy()
        

        pseudo_labels.extend(batch_pseudo_labels)


data['pseudo_label'] = pseudo_labels


In [22]:
data[data['pseudo_label'] == 0]


,source,content,date,cleaned_text,pseudo_label
0,sozcu,"i̇srail , vatandaş Yurt dış seyahat i̇srail ve...",2023-11-03,i srail vatandaş yurt dış seyahat i srail ve...,0
1,euronews,By Euronews yayın tarih 03/11/2023 - 18:22 hab...,2023-11-03,by euronews yayın tarih 03 11 2023 18 22 hab...,0
2,sozcu,"Frans dışişleri bakanlık , i̇srailin düzenleme...",2023-11-03,frans dışişleri bakanlık i srailin düzenleme...,0
3,gazeteduvar,"duvar - Türkiye futbol federasyon , Galatasara...",2023-11-03,duvar türkiye futbol federasyon galatasara...,0
4,euronews,By Euronews yayın tarih 03/11/2023 - 15:18 hab...,2023-11-03,by euronews yayın tarih 03 11 2023 15 18 hab...,0
...,...,...,...,...,...
11238,SABAH,\r\n\r\nİstanbul Fatih'te kuaförlük mesleği ya...,2024-04-22,i̇stanbul fatih te kuaförlük mesleği yapan ab...,0
11239,SABAH,"\r\n\r\nBakırlı konuşmasına ""Bu CHP'nin adayı ...",2024-04-22,bakırlı konuşmasına bu chp nin adayı muhtarl...,0
11240,SABAH,\r\n\r\nBelediye başkan adaylığı açıklandığı g...,2024-04-22,belediye başkan adaylığı açıklandığı günden b...,0
11241,SABAH,\r\n\r\nAK Parti Muğla Büyükşehir Belediye Baş...,2024-04-22,ak parti muğla büyükşehir belediye başkan ada...,0
